In [3]:
#取所有頁面內目前日期所有文章內容
#將取得一頁內容的程式成為函式，並傳出一頁的內容,有無前面連結
from bs4 import BeautifulSoup
import requests
import re
def contentOfOnePage(URL,pttToday):
    headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
    
    #建立detail cookies
    jar = requests.cookies.RequestsCookieJar()
    jar.set('over18','1',domain='www.ptt.cc',path='/')

    response = requests.get(URL,headers=headers,cookies=jar)
    if response.status_code != 200:
        print("連線有問題",response.status_code)
        return None
       
    
    soup = BeautifulSoup(response.text,'html.parser')
    
    
    pre_url_block = soup.find('div','btn-group btn-group-paging')
    pre_url_element = pre_url_block.find_all('a', 'btn wide')[1]
    pre_url = "https://www.ptt.cc" + pre_url_element['href']
    
    r_entElements = soup.find_all('div',{"class":"r-ent"})
    articlesOfpage = []    
    for element in r_entElements:
        pushNum = element.find('div','nrec').get_text().strip()  #取得推文數
        titleElement = element.find('div','title') #取得推文元素
        title = titleElement.get_text().strip() #取得title        
        try: #無作者會是None
            titleLink = titleElement.a['href'] #取得文章連結網址
            author = element.find('div','author').text #取得作者
        except:
            titleLink = '無網址'
            author = "無作者"
        #print(title)
        #print(re.match(r"(\[公告\]|\[協尋\])",title,re.I))
        if re.match(r"(\[公告\]|\[協尋\])",title,re.I): continue  #檢查有前面有[公告]或[協尋]就continue
        #print(element.find('div','date').text.strip())
        if element.find('div','date').text.strip() == pttToday:             
            oneArticle = {}
            try:
                oneArticle['推文數'] = int(pushNum) if pushNum!="" else 0 #如果推文數是空字串則傳回0
            except:
                oneArticle['推文數'] = 100
            
            oneArticle['文章標題'] = title
            oneArticle['文章連結'] = titleLink
            oneArticle['作者'] = author
            articlesOfpage.append(oneArticle)
        else:
            pre_url = None #日期不對就將上一頁設為None
            break
            
    return articlesOfpage, pre_url  #傳出tuple,前面為所有文章,後面為前一頁網址,如果本日文章已經結束,則傳回None

In [5]:
import time
pptUrl = "https://www.ptt.cc/bbs/Gossiping/index.html"
articles = []
#取得目前日期
today = time.strftime('%m/%d')
#符合ptt日期格式
pttToday = today.lstrip('0')
while pptUrl != None: #檢查有沒有日期是否為今日,如果是None就不是今天的日期
    articlesOfpage, pptUrl = contentOfOnePage(pptUrl,pttToday)
    #print(pptUrl)
    articles += articlesOfpage
print('文章數',len(articles))
print('顯示前50筆文章:')
for article in articles[:50]:
    print(article)

文章數 9
顯示前50筆文章:
{'推文數': 0, '文章標題': 'Re: [問卦] 拜耳案是不是台灣人民自己雙標的例子呀!', '文章連結': '/bbs/Gossiping/M.1615253301.A.111.html', '作者': 'gmooshan'}
{'推文數': 0, '文章標題': '[問卦] 台男那麼多時間管理大師 真的有比較好嗎', '文章連結': '/bbs/Gossiping/M.1615253374.A.68F.html', '作者': 'Vincent6964'}
{'推文數': 0, '文章標題': 'Re: [新聞] 王定宇驚爆同居顏若芳 網撻伐：顏色對了', '文章連結': '/bbs/Gossiping/M.1615253380.A.9CE.html', '作者': 'nasa709'}
{'推文數': 0, '文章標題': 'Re: [問卦] 同性戀怎麼沒有被天擇淘汰', '文章連結': '/bbs/Gossiping/M.1615253388.A.C24.html', '作者': 'ZaneTrout'}
{'推文數': 2, '文章標題': '[問卦] 同房不同床的純友誼還有多少人辦得到', '文章連結': '/bbs/Gossiping/M.1615253401.A.8B2.html', '作者': 'ROC92'}
{'推文數': 0, '文章標題': 'Re: [新聞] 快訊／高雄駁二特區凌晨大火\u3000濃煙沖天「', '文章連結': '/bbs/Gossiping/M.1615253437.A.286.html', '作者': 'Gjoy'}
{'推文數': 0, '文章標題': '[問卦] 簽藻礁同意書的人漲電費 你ok嗎', '文章連結': '/bbs/Gossiping/M.1615253463.A.C70.html', '作者': 'rocephin'}
{'推文數': 1, '文章標題': '[新聞] 南部變慶記之都？蘇貞昌：突發情仇殺難免', '文章連結': '/bbs/Gossiping/M.1615253490.A.B8C.html', '作者': 's48692003'}
{'推文數': 0, '文章標題': 'Re: [問卦] 環團廢死同志八卦，台灣四大害？', '文章連結': '/b